# Feature Calculation 
## Input expected to be the output from the CompCalc 
## The function will caluclate material feature like VEC, atomic size difference for each composition 

In [1]:
import pandas as pd
import numpy as np
import math

In [8]:
#prop, eleNum, eles, 
dtdir=r'C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed 0922\CompCal.csv'
dt=pd.read_csv(dtdir,index_col='ID')
eleNum=dt.iloc[:,3:22]
hh  = pd.read_excel('H2.xlsx', index_col="element")
prop=pd.read_csv('chemical_elements.csv')
eles=eleNum.columns
eleNum

,Al,Cr,Mn,Fe,Co,Ni,Cu,Ta,Ti,V,Zr,Nb,Mo,Zn,W,Sn,Nd,Hf,Si
ID,,,,,,,,,,,,,,,,,,,
0,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,0.235294,0.235294,0.000000,0.235294,0.235294,0.000000,0,0.0,0,0,0.0,0.0
1,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,0.235294,0.235294,0.000000,0.235294,0.235294,0.000000,0,0.0,0,0,0.0,0.0
2,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,0.235294,0.235294,0.000000,0.235294,0.235294,0.000000,0,0.0,0,0,0.0,0.0
3,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.238095,0.238095,0.238095,0.000000,0.238095,0.000000,0,0.0,0,0,0.0,0.0
4,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.238095,0.238095,0.238095,0.000000,0.238095,0.000000,0,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.333333,0.333333,0,0.0,0,0,0.0,0.0
744,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.000000,0.303030,0.000000,0.303030,0.303030,0,0.0,0,0,0.0,0.0
745,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.000000,0.285714,0.000000,0.285714,0.285714,0,0.0,0,0,0.0,0.0


In [9]:

VECs=[]
ends=[]
atomSizeDiffs=[]
mixEnths=[]
mixEntps=[]
avgShearMods=[]
avgBulkMods=[]
avgPosRatios=[]
iteElectrons=[]
cohesiveEne=[]
Gammas=[]
localSizeMisMatch=[]
localModulusMisMatch=[]
localElectroMisMatch=[]
avgMelts=[]
def gamma(atomRs):
    avgR=np.average(atomRs)
    maxR=max(atomRs)
    minR=min(atomRs)
    upper=1-np.sqrt(((avgR+minR)**2-avgR**2)/(avgR+minR)**2)
    lower=1-np.sqrt(((avgR+maxR)**2-avgR**2)/(avgR+maxR)**2)
    return upper/lower
def misMatchCal(elementList,weight,propertyName):
    res=0
    propList=[]
    for i in range(len(elementList)):
        propList.append(prop[prop[' symbol'].str.strip()==elementList[i]][propertyName].values[0])
    for e1 in range(len(propList)-1):
        for e2 in range(e1+1,len(propList)):
            res+=(propList[e1]-propList[e2])*weight[e1]*weight[e2]
    return res


for i in range(eleNum.shape[0]):
    avgRadius=0
    avgEle=0
    avgVEC=0
    avg_h=0
    avg_ss=0
    avgSM=0
    avgBM=0
    avgPS=0
    avgIte=0
    avgCoh=0
    avgMelt=0
    atomRsx=[]

    for e1 in range(eles.shape[0]-1):
        element_1 = eles[e1]
        for e2 in range(e1+1,eles.shape[0]):
            element_2 = eles[e2]
            avg_h += 4*hh.loc[element_1,element_2]*eleNum.iloc[i,e2]*eleNum.iloc[i,e1]
    mixEnths.append(avg_h)
    localSizeMisMatch.append(misMatchCal(elementList=eles,weight=eleNum.iloc[i,:],propertyName='radius'))
    localModulusMisMatch.append(misMatchCal(elementList=eles,weight=eleNum.iloc[i,:],propertyName='Shear Modulus (GPa)'))
    localElectroMisMatch.append(misMatchCal(elementList=eles,weight=eleNum.iloc[i,:],propertyName=' electronegativity'))
    for j in range(eles.shape[0]):
        try:
            avg_ss+=math.log(eleNum.iloc[i,j])*eleNum.iloc[i,j]
        except ValueError:
            avg_ss+=0
        try:
            atomRadius=prop[prop[' symbol'].str.strip()==eles[j]]['radius'].values[0]
            atomEleNeg=prop[prop[' symbol'].str.strip()==eles[j]][' electronegativity'].values[0]
            atomVEC=prop[prop[' symbol'].str.strip()==eles[j]]['VEC'].values[0]
            atomShearMod=prop[prop[' symbol'].str.strip()==eles[j]]['Shear Modulus (GPa)'].values[0]
            atomBulkMod=prop[prop[' symbol'].str.strip()==eles[j]]['Bulk Modulus (GPa)'].values[0]
            atomPoisson=prop[prop[' symbol'].str.strip()==eles[j]]['Poissons Ratio'].values[0]
            atomIteElectron=prop[prop[' symbol'].str.strip()==eles[j]]['Number of Itinerant Electrons'].values[0]
            atomCohesive=prop[prop[' symbol'].str.strip()==eles[j]]['Cohesive Energy (eV/atom)'].values[0]
            atomMeltTemp=prop[prop[' symbol'].str.strip()==eles[j]]['meltingPoint'].values[0]
            avgRadius+=atomRadius*eleNum.iloc[i,j]
            avgEle+=atomEleNeg*eleNum.iloc[i,j]
            avgVEC+=atomVEC*eleNum.iloc[i,j]
            avgSM+=atomShearMod*eleNum.iloc[i,j]
            avgBM+=atomBulkMod*eleNum.iloc[i,j]
            avgPS+=atomPoisson*eleNum.iloc[i,j]
            avgIte+=atomIteElectron*eleNum.iloc[i,j]
            avgCoh+=atomCohesive*eleNum.iloc[i,j]
            avgMelt+=atomMeltTemp*eleNum.iloc[i,j]
            if eleNum.iloc[i,j]!=0:
                atomRsx.append(atomRadius)


        except IndexError:
            pass
    atomicSizeDiff=0
    electroNegDiff=0
    for k in range(eles.shape[0]):
        try:
            atomRadius=prop[prop[' symbol'].str.strip()==eles[k]]['radius'].values[0]
            atomEleNeg=prop[prop[' symbol'].str.strip()==eles[k]][' electronegativity'].values[0]
        except IndexError:
            atomRadius=0
            atomEleNeg=0
        atomicSizeDiff+=(1-atomRadius/avgRadius)**2*eleNum.iloc[i,k]
        electroNegDiff+=(atomEleNeg-avgEle)**2*eleNum.iloc[i,k]
    VECs.append(avgVEC)
    ends.append(electroNegDiff**0.5)
    atomSizeDiffs.append((atomicSizeDiff**0.5)*100)
    avgShearMods.append(avgSM)
    avgBulkMods.append(avgBM)
    avgPosRatios.append(avgPS)  
    mixEntps.append(avg_ss*-8.314)
    iteElectrons.append(avgIte)
    cohesiveEne.append(avgCoh)
    Gammas.append(gamma(atomRs=atomRsx))
    avgMelts.append(avgMelt)
dt['VEC']=VECs
dt['ElectDiff']=ends
dt['Atomic Size Diff']=atomSizeDiffs
dt['Mixing Enthalpy']=mixEnths
dt['Mixing Entropy']=mixEntps
dt['AVG Shear Modulus']=avgShearMods
dt['AVG Bulk Modulus']=avgBulkMods
dt['AVG Possion Ratio']=avgPosRatios
dt['Itinerate Electron']=iteElectrons
dt['Cohesive Energy']=cohesiveEne
dt['Gamma']=Gammas
dt['Local Size Mismatch']=localSizeMisMatch
dt['Local Shear Modulus Mismatch']=localModulusMisMatch
dt['Local Electronegativity Mismatch']=localElectroMisMatch
dt['Avg Melting Pt']=avgMelts
#dt.to_csv(testDfloc)

C:\Users\Sir\AppData\Local\Temp\ipykernel_16248\1700199416.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res+=(propList[e1]-propList[e2])*weight[e1]*weight[e2]


In [10]:
dt.to_csv(r'C:\Users\Sir\OneDrive\Univeristy\HEA Analysis\4K06\Code\Data Folder\Processed 0922\FeatureCal.csv')